In [2]:
# Filter Recent Trades
from datetime import datetime as dt ,timedelta
import ccxt
import numpy as np
import pandas as pd
import requests
import time


symbols='BTC-PERP'
end =(dt.now())
end_str = end.strftime('%Y%m%d') ## todays 
end_ts =  int((end.timestamp())) ## interer values before convert iso

start_ts = end -timedelta(seconds=600)
start_ts = int((start_ts.timestamp()))
print(start_ts,end_ts)

# res =exchange.public_get_markets_market_name_trades({'start_time':(int(start_ts)),'end_time':now_ts,'market_name':symbol1})


1646653366 1646653966


In [3]:


def fetch_recent_trades(symbols:'str',start_ts,end_ts,limit=5000): #
    limit=5000 # max 5000 


    
    res = requests.get(f'https://ftx.com/api/markets/{symbols}/trades?limit={limit}&start_time={start_ts}&end_time={end_ts}').json()
    df = pd.DataFrame(res['result'])
    df=df.set_index('time')
    df.index= pd.to_datetime(df.index)
    df =df.sort_index(ascending=True)
    df['value']=df['price'] *df['size']
#     df['chg']=df['price'].pct_change()

    return df



fetch_recent_trades(symbols,start_ts,end_ts)

,id,price,size,side,liquidation,value
time,,,,,,
2022-03-07 11:42:46.559446+00:00,3478254598,38311.0,0.0130,buy,False,498.0430
2022-03-07 11:42:47.661638+00:00,3478254721,38310.0,0.0103,sell,False,394.5930
2022-03-07 11:42:47.661638+00:00,3478254722,38310.0,0.1500,sell,False,5746.5000
2022-03-07 11:42:47.662365+00:00,3478254725,38309.0,0.0141,sell,False,540.1569
2022-03-07 11:42:47.662365+00:00,3478254723,38309.0,0.0100,sell,False,383.0900
...,...,...,...,...,...,...
2022-03-07 11:52:43.097736+00:00,3478284418,38315.0,0.1305,buy,False,5000.1075
2022-03-07 11:52:43.684607+00:00,3478284480,38316.0,0.0105,buy,False,402.3180
2022-03-07 11:52:44.553244+00:00,3478284515,38319.0,0.1600,sell,False,6131.0400


## Monitor Volume ,  Frequency trades

In [13]:
res = requests.get(f'https://ftx.com/api/markets/{symbols}/trades?start_time={start_ts}&end_time={end_ts}').json() # Limit 5000 ?

# df = pd.DataFrame(res['result'])
# df =fetch_recent_trades(symbols,start_ts,end_ts)
df= pd.read_csv('test_trade_data.csv')
df =df.set_index('time')
df.index = pd.to_datetime(df.index)

In [5]:




gb= df.groupby('side').agg({'price':'last','value':'sum','side':'count'})
count_buy = gb.loc['buy','side'] # Count  Buy Trades
count_sell = gb.loc['sell','side'] # Count  Buy Trades
count_sell - count_buy 




print('Values Side Sell',df[df['side']=='sell']['value'].sum(),' $' ,'Count',count_buy )
print('Values Side Buy',df[df['side']=='buy']['value'].sum(),' $','Count',count_sell)
# print(df.tail(10))

gb

Values Side Sell 912850.4399  $ Count 1219
Values Side Buy 702471.9046  $ Count 1445


,price,value,side
side,,,
buy,3.8932,702471.9046,1219
sell,3.8912,912850.4399,1445


# Resample Tickdata


In [8]:
# Liquidation at price 
df[ (df['liquidation'] ==True) ]

,id,price,size,side,liquidation,value
time,,,,,,
2021-11-20 16:17:11.449115+00:00,2517507279,3.8767,102.0,sell,True,395.4234
2021-11-20 16:17:11.449115+00:00,2517507278,3.8767,10.0,sell,True,38.7670
2021-11-20 16:19:52.500238+00:00,2517537602,3.8520,63.0,sell,True,242.6760
2021-11-20 16:21:04.800962+00:00,2517564237,3.8621,197.0,sell,True,760.8337
2021-11-20 16:21:05.385969+00:00,2517564393,3.8624,134.0,sell,True,517.5616
2021-11-20 16:21:05.385969+00:00,2517564394,3.8623,25.0,sell,True,96.5575
2021-11-20 16:21:35.508917+00:00,2517571214,3.8766,133.0,sell,True,515.5878
2021-11-20 16:21:35.508917+00:00,2517571215,3.8764,249.0,sell,True,965.2236
2021-11-20 16:21:36.196863+00:00,2517571282,3.8764,1313.0,sell,True,5089.7132


In [14]:


df_re =df.resample('1T').agg({'price':'last','size':'sum','value':'sum','side':'count','liquidation':'max'})
# df.resample('1T').agg({'id':'last','price':'last','size':'sum','liquidation':'last','value':'sum','side':'count'})#.sort_index(ascending=True)

df_re

,price,size,value,side,liquidation
time,,,,,
2021-11-20 16:15:00+00:00,3.8886,8871.0,34501.9612,48,False
2021-11-20 16:16:00+00:00,3.8828,18102.0,70343.0817,149,False
2021-11-20 16:17:00+00:00,3.8791,17333.0,67269.8734,156,True
2021-11-20 16:18:00+00:00,3.8705,11757.0,45578.9282,136,False
2021-11-20 16:19:00+00:00,3.8547,90853.0,350555.9747,435,True
2021-11-20 16:20:00+00:00,3.8653,63763.0,246127.5477,383,False
2021-11-20 16:21:00+00:00,3.8816,37672.0,146017.7252,299,True
2021-11-20 16:22:00+00:00,3.8740,37363.0,144750.9093,237,False
2021-11-20 16:23:00+00:00,3.8762,8637.0,33486.7171,117,False


In [15]:
grouper = df.groupby([pd.Grouper(freq='1t'), 'side'])
g1= grouper.agg({'price':'last','value':'sum','side':'count','liquidation':'max'})
# df_re =df.resample('1T').agg({'price':'last','size':'sum','value':'sum','side':'count'})

g1.columns = ['price','value','count','liquidation']
g1.tail()
# Cumsum แยก side 

price       value  count  liquidation
time                      side                                        
2021-11-20 16:26:00+00:00 sell  3.8961  19341.7649     57        False
2021-11-20 16:27:00+00:00 buy   3.8843  56484.5549     88        False
                          sell  3.8835  47382.3788    123        False
2021-11-20 16:28:00+00:00 buy   3.8932   9507.0328     34        False
                          sell  3.8912  12780.2517     18        False

In [16]:
df_re.tail()

,price,size,value,side,liquidation
time,,,,,
2021-11-20 16:24:00+00:00,3.8811,7320.0,28398.5026,80,False
2021-11-20 16:25:00+00:00,3.8880,22753.0,88335.5147,159,False
2021-11-20 16:26:00+00:00,3.8995,60090.0,233801.3905,202,False
2021-11-20 16:27:00+00:00,3.8843,26693.0,103866.9337,211,False
2021-11-20 16:28:00+00:00,3.8912,5731.0,22287.2845,52,False


In [17]:
# g1.index

In [18]:
# g1[['value']].plot()

In [19]:
g1

price        value  count  liquidation
time                      side                                         
2021-11-20 16:15:00+00:00 buy   3.8886   19546.2657     35        False
                          sell  3.8881   14955.6955     13        False
2021-11-20 16:16:00+00:00 buy   3.8816   18902.0261     50        False
                          sell  3.8828   51441.0556     99        False
2021-11-20 16:17:00+00:00 buy   3.8854   20988.1030     66        False
                          sell  3.8791   46281.7704     90         True
2021-11-20 16:18:00+00:00 buy   3.8705   16519.7527     58        False
                          sell  3.8710   29059.1755     78        False
2021-11-20 16:19:00+00:00 buy   3.8547   70492.7124    149        False
                          sell  3.8536  280063.2623    286         True
2021-11-20 16:20:00+00:00 buy   3.8675   80894.4601    172        False
                          sell  3.8653  165233.0876    211        False
2021-11-20 16:21:00+00:00 buy   3.8816   83400.7674    160        False
                          sell  3.8806   62616.9578    139         True
2021-11-20 16:22:00+00:00 buy   3.8740   50400.2272    104        False
                          sell  3.8716   94350.6821    133        False
2021-11-20 16:23:00+00:00 buy   3.8762    7850.3377     39        False
                          sell  3.8755   25636.3794     78        False
2021-11-20 16:24:00+00:00 buy   3.8811   11006.6477     43        False
                          sell  3.8816   17391.8549     37        False
2021-11-20 16:25:00+00:00 buy   3.8880   42019.3913     76        False
                          sell  3.8876   46316.1234     83        False
2021-11-20 16:26:00+00:00 buy   3.8995  214459.6256    145        False
                          sell  3.8961   19341.7649     57        False
2021-11-20 16:27:00+00:00 buy   3.8843   56484.5549     88        False
                          sell  3.8835   47382.3788    123        False
2021-11-20 16:28:00+00:00 buy   3.8932    9507.0328     34        False
                          sell  3.8912   12780.2517     18        False

In [20]:
# df.to_csv('test_trade_data.csv')
# df_re.to_csv('test_trade_data_1.csv')
# g1.to_csv('test_trade_data_2.csv')

In [150]:
df= pd.read_csv('test_trade_data.csv')
df

,time,id,price,size,side,liquidation,value
0,2021-11-20 16:15:51.059738+00:00,2517497209,3.8867,210.0,buy,False,816.2070
1,2021-11-20 16:15:51.942115+00:00,2517497299,3.8868,3.0,buy,False,11.6604
2,2021-11-20 16:15:52.195335+00:00,2517497320,3.8868,100.0,buy,False,388.6800
3,2021-11-20 16:15:52.195933+00:00,2517497321,3.8868,121.0,buy,False,470.3028
4,2021-11-20 16:15:52.196304+00:00,2517497322,3.8868,50.0,buy,False,194.3400
...,...,...,...,...,...,...,...
2659,2021-11-20 16:28:15.665414+00:00,2517629586,3.8918,199.0,sell,False,774.4682
2660,2021-11-20 16:28:15.751077+00:00,2517629605,3.8917,345.0,sell,False,1342.6365
2661,2021-11-20 16:28:15.752126+00:00,2517629606,3.8917,612.0,sell,False,2381.7204
2662,2021-11-20 16:28:15.752513+00:00,2517629607,3.8917,458.0,sell,False,1782.3986
